In [5]:
import pickle

In [11]:
import pickle
import sys
import nltk
import torch.nn as nn
import numpy as np
nltk.download('punkt')
from gensim.models import Word2Vec
sys.path.insert(0, '..')
import torch

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def save_lists_to_file(path):
    # Open the pickle file in binary write mode
    with open(path, 'rb') as f:
    # Load each list from the file
        NE = pickle.load(f)
        EEG_segments = pickle.load(f)
        Classes = pickle.load(f)

    return NE, EEG_segments, Classes

In [59]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=50, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in Word_Labels_List:
        Embedded_Word_labels.append(word_embeddings[word])

    return word_embeddings, Embedded_Word_labels

In [14]:
train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [15]:
train_NE, train_EEG_segments, train_Classes = save_lists_to_file(train_path)
test_NE, test_EEG_segments, test_Classes = save_lists_to_file(test_path)

In [23]:
len(train_EEG_segments[0][0])

2

In [47]:
for word in train_NE:
    if len(word) > 6:
        print(word)

['Florida', 'State', 'Normal', 'and', 'Industrial', 'College', 'and']


In [36]:
len(train_EEG_segments[0])

61

In [62]:
def flatten_EEG_labels(NE_list, EEG_list):

    list_of_eeg_segments = []
    list_of_word_labels = []
    for i in range(len(NE_list)):
        Named_Entity = NE_list[i]
        Named_Entity_EEG_Segments = EEG_list[i]
        for j in range(len(Named_Entity)):
            word = Named_Entity[j]
            for EEG_Segments in range(len(Named_Entity_EEG_Segments)):
                EEG_Segment = Named_Entity_EEG_Segments[EEG_Segments][j]
                list_of_eeg_segments.append(EEG_Segment)
                list_of_word_labels.append(word)

    return list_of_eeg_segments, list_of_word_labels




In [63]:


list_of_eeg_segments, list_of_word_labels = flatten_EEG_labels(train_NE, train_EEG_segments)

25865

In [16]:

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [60]:
#embeddings
#Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)

word_embeddings, list_of_embeddings = create_word_label_embeddings(list_of_word_labels)


Number of word embeddings: 683


In [68]:
def create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels):
    """
    This function takes in the EEG word level embeddings and the word labels and returns a dataloader

    :param EEG_word_level_embeddings: The EEG segments of the associated textual information
    :param Embedded_Word_labels: The word embeddings of the associated textual information
    :return trainloader: The dataloader for the EEG word level embeddings and the word labels
    """

    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.max(
        np.abs(EEG_word_level_embeddings))

    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float)
    float_tensor = float_tensor.unsqueeze(1)

    # Sanity check
    print(torch.isnan(float_tensor).any())

    train_data = []
    for i in range(len(float_tensor)):
        train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader

In [69]:
train_loader = create_dataloader(list_of_eeg_segments, list_of_embeddings)

tensor(False)
